# How to show a BPMN diagram in Juypter

BPMN means "Business Process Model and Notation" and is standard format for these diagrams, used both in abstract process modelling and process automation. For a quick introduction check the [BPMN tutorial](https://camunda.com/bpmn/) (by [Camunda](https://camunda.com)) or this [blog post](https://medium.com/@ruslanfg/camunda-developer-friendly-bpms-21c211903917) (by [Ruslan Gainutdinov](https://ruslan.org/))

Camunda is a company that offers consulting and software solutions for process modelling and automation. Some of these software are open source like the:
 * [Camunda Modeler](https://github.com/camunda/camunda-modeler), a Desktop gui for process modelling, written in Java
 * [bpmn-js](https://github.com/bpmn-io/bpmn-js), a web viewer and editor for BPMN diagrams, written in Javascript to be embedded in other applications
 
Some are open source with paid premium features, like the:
 * [Camunda BPM platform](https://github.com/camunda/camunda-bpm-platform), which includes a process automation engine with a powerful REST API and some web tools like the [tasklist](https://camunda.com/products/camunda-bpm/tasklist/) (assign tasks to users, check pending tasks, fulfill manual tasks) and [cockpit](https://camunda.com/products/camunda-bpm/cockpit/) (analyze and troubleshoot running process instances). 
 
They also offer paid products like [Caewemo](https://camunda.com/products/camunda-bpm/cawemo/) (a collaborative online version of the modeler) and [Optimize](https://camunda.com/products/camunda-bpm/optimize/) (an analytics tool)

#### Motivation (my short story with Camunda)
Camunda BPM platform is designed to be **developer friendly**. Which means to build anything useful developers are needed, it was not built to be a no-code solution. Frindly here means it is based on open formats and APIs.

I have been studying Camunda for about a month. I tried to follow the tutorials and recreate the [invoce demo application](https://www.youtube.com/watch?v=2XeTJQfz_YQ), which is shipped with the platform. However I have no Java experience, and I struggled a lot. I also discovered that some features I needed are not available on the community version of the [cockpit](https://camunda.com/products/camunda-bpm/cockpit/), only in the paid version. I also found Camunda docs to be a bit dry.

So I heard from a colleague that most things I need are available on the [REST API](https://docs.camunda.org/manual/7.14/reference/rest/), and decided to dive into it using python and Jupyter notebooks, because I am very confortable to try new things in this environment.

I started doing basic [process definition](https://docs.camunda.org/manual/7.14/reference/rest/process-definition/) stuff with the demo invoice application, like:
 * List of deployed process <br>
     &emsp; GET /process-definition
 * Basic statistics of one process (number of running stances, incidents and failures of each process step)<br>
     &emsp; GET /process-definition/key/{key}/statistics
 * Download the process diagram (xml of the .bpmn file)<br>
     &emsp; GET /process-definition/key/{key}/diagram
     
After I downloaded the diagram xml, I wanted to visualize it in Jupyter notebook. I was confident I would find a solution online, but I didn't. I found for an [extesion for jupyterlab](https://github.com/PhE/jupyterlab_bpmn) and [another for R](https://github.com/bergant/bpmn), but nothing for python Jupyter notebooks.

Then I had an idea to blend python, HTML, javacript and [bpmn-js](https://github.com/bpmn-io/bpmn-js) a make a function to render the diagram. I barely know javascript and struggled for 2 days reading many tutorials and online resources, but couldn't make it work.

Then I found [bpmn-visualization-js](https://github.com/process-analytics/bpmn-visualization-js), an alternative viewer to [bpmn-js](https://github.com/bpmn-io/bpmn-js) and after a few minutes, it was working.

### 1. Imports and the rendering function code
 * The fuction take as input a string containing the full .bpmn xml content.<br>
 * Then it remove newlines (/n) from the string. I added this because the new lines were breaking the html/javascript code. It is possible that other escape characters will cause the some issue, but it didn't happen with the diagrams I tested.
 * It generates the html div name to put the bpmn viewer. I included a random number in the string because using the function in more than one cell, it was always rendering in the first jupyter output cell. With the random number element the div will have a "unique" name (although there is tiny collision riskb
 * Then there is html code, which is filled with both the {bpmn_div} name and {bpmn_xml} using python f-string. Note that:
   * It load the viewr from a CDN
   * The full bpmn xml become part of the generated html/javacript code
 * Finally, the function execute the html/javascript code to render the diagram, using Python.display.HTML function

In [1]:
from IPython.display import HTML
from random import randint

In [2]:
def bpmn(bpmn_xml):
    bpmn_xml = bpmn_xml.replace('\n', '')
    bmpn_div = f'bpmn-container-{randint(0, 1000000)}'
    html_code = f'''
    <div id='{bmpn_div}'</div>
    <script src="https://cdn.jsdelivr.net/npm/bpmn-visualization@0.9.0/dist/bpmn-visualization.min.js"></script>
    <script>
        var bpmn_xml = '{bpmn_xml}';
        const bpmnVisualization = new bpmnvisu.BpmnVisualization(document.getElementById('{bmpn_div}'));
        bpmnVisualization.load(bpmn_xml);
    </script>
      '''
    return HTML(html_code)

### 2. A full BPMN xml in a python string
 * Created with [Camunda Modeler](https://github.com/camunda/camunda-modeler)
 * Saved as .bpmn file
 * Opened in a text editor,
 * Pasted here in a multi-line raw python string

In [3]:
my_bpmn_xml = r'''<?xml version="1.0" encoding="UTF-8"?>
<bpmn:definitions xmlns:bpmn="http://www.omg.org/spec/BPMN/20100524/MODEL" xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI" xmlns:dc="http://www.omg.org/spec/DD/20100524/DC" xmlns:di="http://www.omg.org/spec/DD/20100524/DI" id="Definitions_0xm3zt1" targetNamespace="http://bpmn.io/schema/bpmn" exporter="Camunda Modeler" exporterVersion="4.4.0">
  <bpmn:process id="Process_122ys79" isExecutable="true">
    <bpmn:startEvent id="StartEvent_1">
      <bpmn:outgoing>Flow_1j4w0ym</bpmn:outgoing>
    </bpmn:startEvent>
    <bpmn:task id="Activity_01hik5s" name="hello world">
      <bpmn:incoming>Flow_1j4w0ym</bpmn:incoming>
      <bpmn:outgoing>Flow_0oukpbm</bpmn:outgoing>
    </bpmn:task>
    <bpmn:sequenceFlow id="Flow_1j4w0ym" sourceRef="StartEvent_1" targetRef="Activity_01hik5s" />
    <bpmn:endEvent id="Event_1k63k9m">
      <bpmn:incoming>Flow_0oukpbm</bpmn:incoming>
    </bpmn:endEvent>
    <bpmn:sequenceFlow id="Flow_0oukpbm" sourceRef="Activity_01hik5s" targetRef="Event_1k63k9m" />
  </bpmn:process>
  <bpmndi:BPMNDiagram id="BPMNDiagram_1">
    <bpmndi:BPMNPlane id="BPMNPlane_1" bpmnElement="Process_122ys79">
      <bpmndi:BPMNEdge id="Flow_1j4w0ym_di" bpmnElement="Flow_1j4w0ym">
        <di:waypoint x="215" y="117" />
        <di:waypoint x="270" y="117" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNEdge id="Flow_0oukpbm_di" bpmnElement="Flow_0oukpbm">
        <di:waypoint x="370" y="117" />
        <di:waypoint x="432" y="117" />
      </bpmndi:BPMNEdge>
      <bpmndi:BPMNShape id="_BPMNShape_StartEvent_2" bpmnElement="StartEvent_1">
        <dc:Bounds x="179" y="99" width="36" height="36" />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Activity_01hik5s_di" bpmnElement="Activity_01hik5s">
        <dc:Bounds x="270" y="77" width="100" height="80" />
      </bpmndi:BPMNShape>
      <bpmndi:BPMNShape id="Event_1k63k9m_di" bpmnElement="Event_1k63k9m">
        <dc:Bounds x="432" y="99" width="36" height="36" />
      </bpmndi:BPMNShape>
    </bpmndi:BPMNPlane>
  </bpmndi:BPMNDiagram>
</bpmn:definitions>'''

### 3. Rendering the BPMN diagram

In [4]:
bpmn(my_bpmn_xml)